In [ ]:
!pip install hierarchicalforecast statsforecast datasetsforecast

In [392]:
import pandas as pd
import numpy as np
import os

from datasetsforecast.hierarchical import HierarchicalData

from hierarchicalforecast.core import HierarchicalReconciliation
from hierarchicalforecast.methods import BottomUp, TopDown, MiddleOut, MinTrace, ERM

from statsforecast.core import StatsForecast
from statsforecast.models import AutoARIMA, Naive

from hierarchicalforecast.evaluation import HierarchicalEvaluation

from sklearn.metrics import mean_squared_error as mse

import matplotlib.pyplot as plt
%matplotlib inline

In [393]:
plt.rcParams.update({'figure.figsize':(10,5)})

In [394]:
def rmse(x,y):
    return np.round(np.sqrt(mse(x.values,y.values)),4)

# Read the Data and we Begin

In [396]:
caldf = pd.read_csv("C:/Users/bhara/Downloads/calendar.csv",parse_dates=['date'])
df = pd.read_csv("C:/Users/bhara/Downloads/sales_train_evaluation.csv/sales_train_evaluation.csv")
                    

In [397]:
caldf.head()

,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [398]:
df.head()

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
0,HOBBIES_1_001_CA_1_evaluation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,4,0,0,0,0,3,3,0,1
1,HOBBIES_1_002_CA_1_evaluation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,1,2,1,1,0,0,0,0,0
2,HOBBIES_1_003_CA_1_evaluation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,2,0,0,0,2,3,0,1
3,HOBBIES_1_004_CA_1_evaluation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,1,0,4,0,1,3,0,2,6
4,HOBBIES_1_005_CA_1_evaluation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,2,1,0,0,2,1,0


In [399]:
caldf = caldf.loc[:,["date","wm_yr_wk","d"]]

In [400]:
caldf.shape

(1969, 3)

## For one item we gonna do HierTS

In [402]:
df[df['item_id']== 'FOODS_3_819']


,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1932,d_1933,d_1934,d_1935,d_1936,d_1937,d_1938,d_1939,d_1940,d_1941
3040,FOODS_3_819_CA_1_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_1,CA,3,7,4,5,...,1,2,6,2,1,2,1,4,2,5
6089,FOODS_3_819_CA_2_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_2,CA,7,6,2,0,...,0,3,1,0,4,4,2,1,1,0
9138,FOODS_3_819_CA_3_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_3,CA,8,9,5,4,...,1,0,2,1,1,1,0,1,1,2
12187,FOODS_3_819_CA_4_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,4,4,3,1,...,5,0,0,0,1,4,0,1,0,1
15236,FOODS_3_819_TX_1_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_1,TX,2,1,0,0,...,0,1,0,2,0,0,0,2,2,0
18285,FOODS_3_819_TX_2_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_2,TX,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
21334,FOODS_3_819_TX_3_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_3,TX,9,0,3,0,...,0,0,0,0,0,0,0,0,1,1
24383,FOODS_3_819_WI_1_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_1,WI,8,3,3,1,...,1,5,2,1,0,1,2,7,0,7
27432,FOODS_3_819_WI_2_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_2,WI,1,0,0,0,...,2,2,1,2,3,3,0,1,4,0
30481,FOODS_3_819_WI_3_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_3,WI,14,11,5,6,...,2,1,2,4,1,6,4,4,1,1


In [403]:
df=df.loc[df["item_id"]== 'FOODS_3_819']

## Melting it down from columns to rows

In [405]:
df_t = df.melt(id_vars=['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id'])
df_t.head(20)

,id,item_id,dept_id,cat_id,store_id,state_id,variable,value
0,FOODS_3_819_CA_1_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_1,CA,d_1,3
1,FOODS_3_819_CA_2_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_2,CA,d_1,7
2,FOODS_3_819_CA_3_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_3,CA,d_1,8
3,FOODS_3_819_CA_4_evaluation,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,d_1,4
4,FOODS_3_819_TX_1_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_1,TX,d_1,2
5,FOODS_3_819_TX_2_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_2,TX,d_1,0
6,FOODS_3_819_TX_3_evaluation,FOODS_3_819,FOODS_3,FOODS,TX_3,TX,d_1,9
7,FOODS_3_819_WI_1_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_1,WI,d_1,8
8,FOODS_3_819_WI_2_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_2,WI,d_1,1
9,FOODS_3_819_WI_3_evaluation,FOODS_3_819,FOODS_3,FOODS,WI_3,WI,d_1,14


In [406]:
df_t.drop(columns="id",inplace=True)

In [407]:
df_t.head()

,item_id,dept_id,cat_id,store_id,state_id,variable,value
0,FOODS_3_819,FOODS_3,FOODS,CA_1,CA,d_1,3
1,FOODS_3_819,FOODS_3,FOODS,CA_2,CA,d_1,7
2,FOODS_3_819,FOODS_3,FOODS,CA_3,CA,d_1,8
3,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,d_1,4
4,FOODS_3_819,FOODS_3,FOODS,TX_1,TX,d_1,2


In [408]:
caldf.head()

,date,wm_yr_wk,d
0,2011-01-29,11101,d_1
1,2011-01-30,11101,d_2
2,2011-01-31,11101,d_3
3,2011-02-01,11101,d_4
4,2011-02-02,11101,d_5


## Merging with cal df on d = variable to get date and rest 

In [410]:
newdf = df_t.merge(caldf,left_on='variable',right_on='d',how='left')
newdf.head()

,item_id,dept_id,cat_id,store_id,state_id,variable,value,date,wm_yr_wk,d
0,FOODS_3_819,FOODS_3,FOODS,CA_1,CA,d_1,3,2011-01-29,11101,d_1
1,FOODS_3_819,FOODS_3,FOODS,CA_2,CA,d_1,7,2011-01-29,11101,d_1
2,FOODS_3_819,FOODS_3,FOODS,CA_3,CA,d_1,8,2011-01-29,11101,d_1
3,FOODS_3_819,FOODS_3,FOODS,CA_4,CA,d_1,4,2011-01-29,11101,d_1
4,FOODS_3_819,FOODS_3,FOODS,TX_1,TX,d_1,2,2011-01-29,11101,d_1


In [411]:
newdf.rename(columns={"value":"sales_qty"},inplace=True)
df=newdf.loc[newdf['date']>='2014-01-01',['date', 'store_id', 'sales_qty']]
df["State_ID"] = df["store_id"].str[:2]

df.head()

,date,store_id,sales_qty,State_ID
10680,2014-01-01,CA_1,3,CA
10681,2014-01-01,CA_2,0,CA
10682,2014-01-01,CA_3,7,CA
10683,2014-01-01,CA_4,0,CA
10684,2014-01-01,TX_1,0,TX


### Long form matrix

In [413]:
df_ind = df.groupby(['date','store_id'])[['sales_qty']].sum()


In [414]:
df_ind.reset_index(inplace=True)


In [415]:
df_ind

,date,store_id,sales_qty
0,2014-01-01,CA_1,3
1,2014-01-01,CA_2,0
2,2014-01-01,CA_3,7
3,2014-01-01,CA_4,0
4,2014-01-01,TX_1,0
...,...,...,...
8725,2016-05-22,TX_2,0
8726,2016-05-22,TX_3,1
8727,2016-05-22,WI_1,7
8728,2016-05-22,WI_2,0


In [416]:
df_ind.columns=["ds","unique_id","sales"]

In [417]:
df_ind

,ds,unique_id,sales
0,2014-01-01,CA_1,3
1,2014-01-01,CA_2,0
2,2014-01-01,CA_3,7
3,2014-01-01,CA_4,0
4,2014-01-01,TX_1,0
...,...,...,...
8725,2016-05-22,TX_2,0
8726,2016-05-22,TX_3,1
8727,2016-05-22,WI_1,7
8728,2016-05-22,WI_2,0


In [418]:
df_sta = df.groupby(['date','State_ID'])[['sales_qty']].sum()
df_sta.reset_index(inplace=True)
df_sta.columns=['ds','unique_id','sales']

df_sta


,ds,unique_id,sales
0,2014-01-01,CA,10
1,2014-01-01,TX,2
2,2014-01-01,WI,4
3,2014-01-02,CA,5
4,2014-01-02,TX,1
...,...,...,...
2614,2016-05-21,TX,3
2615,2016-05-21,WI,5
2616,2016-05-22,CA,8
2617,2016-05-22,TX,1


In [419]:
df_tot = df.groupby(['date'])[['sales_qty']].sum()
df_tot.reset_index(inplace=True)
df_tot

,date,sales_qty
0,2014-01-01,16
1,2014-01-02,8
2,2014-01-03,24
3,2014-01-04,27
4,2014-01-05,23
...,...,...
868,2016-05-18,22
869,2016-05-19,9
870,2016-05-20,21
871,2016-05-21,12


In [420]:
df_tot['unique_id']='Total'
df_tot.columns = ['ds', 'sales', 'unique_id' ]

In [421]:
df_tot

,ds,sales,unique_id
0,2014-01-01,16,Total
1,2014-01-02,8,Total
2,2014-01-03,24,Total
3,2014-01-04,27,Total
4,2014-01-05,23,Total
...,...,...,...
868,2016-05-18,22,Total
869,2016-05-19,9,Total
870,2016-05-20,21,Total
871,2016-05-21,12,Total


In [422]:
fin_df = pd.concat([df_ind,df_sta,df_tot],axis=0)
print(df_ind.shape,df_sta.shape,df_tot.shape,fin_df.shape)

(8730, 3) (2619, 3) (873, 3) (12222, 3)


In [423]:
fin_df.head()

,ds,unique_id,sales
0,2014-01-01,CA_1,3
1,2014-01-01,CA_2,0
2,2014-01-01,CA_3,7
3,2014-01-01,CA_4,0
4,2014-01-01,TX_1,0


In [424]:
fin_df['unique_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3', 'CA', 'TX', 'WI', 'Total'], dtype=object)

In [425]:
fin_df['ds'].min()

Timestamp('2014-01-01 00:00:00')

In [426]:
fin_df['ds'].max()

Timestamp('2016-05-22 00:00:00')

In [430]:
fin_df['ds'].max()-fin_df['ds'].min()

Timedelta('872 days 00:00:00')

In [431]:
print(fin_df['ds'].dtype)

datetime64[ns]


In [432]:
xset = set(fin_df.unique_id)
fin_df.columns = ["ds","unique_id","y"]
fin_df.head(-1)

,ds,unique_id,y
0,2014-01-01,CA_1,3
1,2014-01-01,CA_2,0
2,2014-01-01,CA_3,7
3,2014-01-01,CA_4,0
4,2014-01-01,TX_1,0
...,...,...,...
867,2016-05-17,Total,11
868,2016-05-18,Total,22
869,2016-05-19,Total,9
870,2016-05-20,Total,21


Create Summing Matrix

In [434]:
xset

{'CA',
 'CA_1',
 'CA_2',
 'CA_3',
 'CA_4',
 'TX',
 'TX_1',
 'TX_2',
 'TX_3',
 'Total',
 'WI',
 'WI_1',
 'WI_2',
 'WI_3'}

In [575]:
c=0
for x in xset:
    if "_" in x:
        c+=1
print(c)
S = np.zeros((len(xset),c))

10


In [577]:
S

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [579]:
Sums = pd.DataFrame(S,index=['Total', 'CA','TX','WI','CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3'],
                    columns=['CA_1','CA_2','CA_3','CA_4','TX_1','TX_2','TX_3','WI_1','WI_2','WI_3'])
Sums

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
Total,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TX,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
WI,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA_3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
CA_4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TX_1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
TX_2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [581]:
Sums.loc["Total"]=1
for x in Sums.columns:
    Sums.loc[x][x]=1
Sums.loc['CA'][['CA_1', 'CA_2', 'CA_3', 'CA_4']]=1
Sums.loc['TX'][['TX_1', 'TX_2', 'TX_3']]=1
Sums.loc['WI'][['WI_1', 'WI_2', 'WI_3']]=1
Sums=Sums.astype(int)

C:\Users\bhara\AppData\Local\Temp\ipykernel_17352\1887744443.py:3: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  Sums.loc[x][x]=1
C:\Users\bhara\AppData\Local\Temp\ipykernel_17352\1887744443.py:4: FutureWarning: ChainedAssignmentError: behav

In [583]:
Sums

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
Total,1,1,1,1,1,1,1,1,1,1
CA,1,1,1,1,0,0,0,0,0,0
TX,0,0,0,0,1,1,1,0,0,0
WI,0,0,0,0,0,0,0,1,1,1
CA_1,1,0,0,0,0,0,0,0,0,0
CA_2,0,1,0,0,0,0,0,0,0,0
CA_3,0,0,1,0,0,0,0,0,0,0
CA_4,0,0,0,1,0,0,0,0,0,0
TX_1,0,0,0,0,1,0,0,0,0,0
TX_2,0,0,0,0,0,1,0,0,0,0


In [553]:
tags = {}

tags['Country'] = np.array(['Total'], dtype=object)
tags['Country/State'] = np.array(['CA', 'TX', 'WI'], dtype=object)
tags['Country/State/Store'] = np.array(['CA_1', 'CA_2', 'CA_3', 'CA_4',  
                                        'TX_1', 'TX_2', 'TX_3',
                                        'WI_1', 'WI_2', 'WI_3'], dtype=object)
tags

{'Country': array(['Total'], dtype=object),
 'Country/State': array(['CA', 'TX', 'WI'], dtype=object),
 'Country/State/Store': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object)}

## Next involves Baseline forecast prediction

### For each unique id, last 7 values stored in x_test --> x_train is rest of them from fin_df --> uniq id becomes index for both


In [556]:
horizon = 7

x_test = fin_df.groupby("unique_id").tail(horizon)
x_train = fin_df.drop(x_test.index)
x_train = x_train.set_index("unique_id")
x_test = x_test.set_index("unique_id")

In [558]:
x_train

,ds,y
unique_id,,
CA_1,2014-01-01,3
CA_2,2014-01-01,0
CA_3,2014-01-01,7
CA_4,2014-01-01,0
TX_1,2014-01-01,0
...,...,...
Total,2016-05-11,13
Total,2016-05-12,9
Total,2016-05-13,12


In [560]:
x_test

,ds,y
unique_id,,
CA_1,2016-05-16,2
CA_2,2016-05-16,0
CA_3,2016-05-16,1
CA_4,2016-05-16,0
TX_1,2016-05-16,2
...,...,...
Total,2016-05-18,22
Total,2016-05-19,9
Total,2016-05-20,21


## AutoARIMA preds

In [563]:
# Stasfrorecast needs unique_id, ds, y column names only
x_train.columns

Index(['ds', 'y'], dtype='object')

In [565]:
x_train=x_train.reset_index()

In [490]:
x_train

,unique_id,ds,y
0,CA_1,2014-01-01,3
1,CA_2,2014-01-01,0
2,CA_3,2014-01-01,7
3,CA_4,2014-01-01,0
4,TX_1,2014-01-01,0
...,...,...,...
12084,Total,2016-05-11,13
12085,Total,2016-05-12,9
12086,Total,2016-05-13,12
12087,Total,2016-05-14,14


In [492]:
fc = StatsForecast(models=[AutoARIMA(season_length=7)],freq="D",n_jobs=-1)
x_hat = fc.forecast(df=x_train,h=horizon)

x_hat

,unique_id,ds,AutoARIMA
0,CA,2016-05-16,6.236943
1,CA,2016-05-17,6.033696
2,CA,2016-05-18,6.033696
3,CA,2016-05-19,6.033696
4,CA,2016-05-20,6.033696
...,...,...,...
93,WI_3,2016-05-18,1.885268
94,WI_3,2016-05-19,1.885268
95,WI_3,2016-05-20,1.885268
96,WI_3,2016-05-21,1.885268


In [494]:
x_test.columns

Index(['ds', 'y'], dtype='object')

In [498]:
x_test.reset_index(inplace=True)
x_test

,unique_id,ds,y
0,CA_1,2016-05-16,2
1,CA_2,2016-05-16,0
2,CA_3,2016-05-16,1
3,CA_4,2016-05-16,0
4,TX_1,2016-05-16,2
...,...,...,...
93,Total,2016-05-18,22
94,Total,2016-05-19,9
95,Total,2016-05-20,21
96,Total,2016-05-21,12


In [500]:
xnew = pd.merge(x_test,x_hat,on=['ds','unique_id'])
xnew.head()

,unique_id,ds,y,AutoARIMA
0,CA_1,2016-05-16,2,1.854500
1,CA_2,2016-05-16,0,1.576152
2,CA_3,2016-05-16,1,1.597794
3,CA_4,2016-05-16,0,0.878472
4,TX_1,2016-05-16,2,0.378133


In [502]:
xnew['unique_id'].unique()

array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
       'WI_2', 'WI_3', 'CA', 'TX', 'WI', 'Total'], dtype=object)

In [510]:
print("Overall RMSE",rmse(xnew['y'],xnew['AutoARIMA']))

Overall RMSE 2.1537


In [512]:
tags

{'Country': array(['Total'], dtype=object),
 'Country/State': array(['CA', 'TX', 'WI'], dtype=object),
 'Country/State/Store': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object)}

In [652]:
for k in tags.keys():
    x_rows = xnew[xnew['unique_id'].isin(tags[k])]
    print(x_rows)
    print(k+" rmse : "+ str(rmse(x_rows['y'],x_rows['AutoARIMA'])))

   unique_id         ds   y  AutoARIMA
91     Total 2016-05-16  12  14.092568
92     Total 2016-05-17  11  14.102201
93     Total 2016-05-18  22  14.102201
94     Total 2016-05-19   9  14.102201
95     Total 2016-05-20  21  14.102201
96     Total 2016-05-21  12  14.102201
97     Total 2016-05-22  17  14.102201
Country rmse : 4.8227
   unique_id         ds   y  AutoARIMA
70        CA 2016-05-16   3   6.236943
71        TX 2016-05-16   2   1.635103
72        WI 2016-05-16   7   6.399759
73        CA 2016-05-17   7   6.033696
74        TX 2016-05-17   0   1.731954
75        WI 2016-05-17   4   6.139512
76        CA 2016-05-18  11   6.033696
77        TX 2016-05-18   1   1.578736
78        WI 2016-05-18  10   6.176647
79        CA 2016-05-19   3   6.033696
80        TX 2016-05-19   0   1.719678
81        WI 2016-05-19   6   6.069265
82        CA 2016-05-20   7   6.033696
83        TX 2016-05-20   2   1.713701
84        WI 2016-05-20  12   6.287642
85        CA 2016-05-21   4   6.033696
86 

# BottomUp

* forecast each of the disaggregated series at the lowest level of the hierarchy

* add up to the higher levels

In [519]:
reconcilers = [BottomUp()]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)


In [521]:
x_hat

,unique_id,ds,AutoARIMA
0,CA,2016-05-16,6.236943
1,CA,2016-05-17,6.033696
2,CA,2016-05-18,6.033696
3,CA,2016-05-19,6.033696
4,CA,2016-05-20,6.033696
...,...,...,...
93,WI_3,2016-05-18,1.885268
94,WI_3,2016-05-19,1.885268
95,WI_3,2016-05-20,1.885268
96,WI_3,2016-05-21,1.885268


In [585]:
Sums

,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
Total,1,1,1,1,1,1,1,1,1,1
CA,1,1,1,1,0,0,0,0,0,0
TX,0,0,0,0,1,1,1,0,0,0
WI,0,0,0,0,0,0,0,1,1,1
CA_1,1,0,0,0,0,0,0,0,0,0
CA_2,0,1,0,0,0,0,0,0,0,0
CA_3,0,0,1,0,0,0,0,0,0,0
CA_4,0,0,0,1,0,0,0,0,0,0
TX_1,0,0,0,0,1,0,0,0,0,0
TX_2,0,0,0,0,0,1,0,0,0,0


In [525]:
tags

{'Country': array(['Total'], dtype=object),
 'Country/State': array(['CA', 'TX', 'WI'], dtype=object),
 'Country/State/Store': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object)}

In [735]:
reconcilers = [BottomUp()]
hrec = HierarchicalReconciliation(reconcilers = reconcilers)

In [589]:
Sums = Sums.reset_index().rename(columns={'index': 'unique_id'})

In [738]:
Sums

,unique_id,CA_1,CA_2,CA_3,CA_4,TX_1,TX_2,TX_3,WI_1,WI_2,WI_3
0,Total,1,1,1,1,1,1,1,1,1,1
1,CA,1,1,1,1,0,0,0,0,0,0
2,TX,0,0,0,0,1,1,1,0,0,0
3,WI,0,0,0,0,0,0,0,1,1,1
4,CA_1,1,0,0,0,0,0,0,0,0,0
5,CA_2,0,1,0,0,0,0,0,0,0,0
6,CA_3,0,0,1,0,0,0,0,0,0,0
7,CA_4,0,0,0,1,0,0,0,0,0,0
8,TX_1,0,0,0,0,1,0,0,0,0,0
9,TX_2,0,0,0,0,0,1,0,0,0,0


In [742]:
#x_hat_rec = hrec.reconcile(x_hat, x_train, S, tags)
x_hat_rec_bu = hrec.reconcile(x_hat, Sums, tags)

In [744]:
x_hat_rec_bu

,unique_id,ds,AutoARIMA,AutoARIMA/BottomUp
0,Total,2016-05-16,14.092568,14.021916
1,Total,2016-05-17,14.102201,13.279506
2,Total,2016-05-18,14.102201,13.362153
3,Total,2016-05-19,14.102201,13.769708
4,Total,2016-05-20,14.102201,13.703248
...,...,...,...,...
93,WI_3,2016-05-18,1.885268,1.885268
94,WI_3,2016-05-19,1.885268,1.885268
95,WI_3,2016-05-20,1.885268,1.885268
96,WI_3,2016-05-21,1.885268,1.885268


In [746]:
def mse(y,yhat):
    return np.mean((y-yhat)**2)
evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_hat_df=x_hat_rec,Y_test_df=x_test,tags=tags,benchmark="AutoARIMA")

C:\Users\bhara\anaconda3\Lib\site-packages\hierarchicalforecast\evaluation.py:370: FutureWarning: This class (HierarchicalEvaluation) will be deprecated in future releases. Please use the `hierarchicalforecast.evaluate` function instead.
  warnings.warn(


,level,metric,AutoARIMA,AutoARIMA/BottomUp
0,Country,mse-scaled,1.0,1.056089
1,Country/State,mse-scaled,1.0,1.073777
2,Country/State/Store,mse-scaled,1.0,1.000000
3,Overall,mse-scaled,1.0,1.039537


In [748]:
xmat = pd.merge(left = x_test, right = x_hat_rec, on = ['ds', 'unique_id'])
xmat.head(3)

,ds,unique_id,y,AutoARIMA,AutoARIMA/BottomUp
0,2016-05-16,CA_1,2,1.854500,1.854500
1,2016-05-16,CA_2,0,1.576152,1.576152
2,2016-05-16,CA_3,1,1.597794,1.597794


In [750]:
mse1 = rmse(xmat['y'], xmat['AutoARIMA'])
mse2 = rmse(xmat['y'], xmat['AutoARIMA/BottomUp'])
print('overall rmse: ' + str(mse1) + ' -> ' + str(mse2))

overall rmse: 2.1537 -> 2.1959


In [752]:
xmat.tail(10)

,ds,unique_id,y,AutoARIMA,AutoARIMA/BottomUp
88,2016-05-22,CA,8,6.033696,6.063674
89,2016-05-22,TX,1,1.725936,1.959353
90,2016-05-22,WI,8,6.696967,6.471231
91,2016-05-16,Total,12,14.092568,14.021916
92,2016-05-17,Total,11,14.102201,13.279506
93,2016-05-18,Total,22,14.102201,13.362153
94,2016-05-19,Total,9,14.102201,13.769708
95,2016-05-20,Total,21,14.102201,13.703248
96,2016-05-21,Total,12,14.102201,14.402036
97,2016-05-22,Total,17,14.102201,14.494257


In [664]:
np.unique(xmat['unique_id'])

array(['CA', 'CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX', 'TX_1', 'TX_2', 'TX_3',
       'Total', 'WI', 'WI_1', 'WI_2', 'WI_3'], dtype=object)

In [678]:
tags

{'Country': array(['Total'], dtype=object),
 'Country/State': array(['CA', 'TX', 'WI'], dtype=object),
 'Country/State/Store': array(['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1',
        'WI_2', 'WI_3'], dtype=object)}

In [754]:
xmat[xmat['unique_id']=='CA_1']


,ds,unique_id,y,AutoARIMA,AutoARIMA/BottomUp
0,2016-05-16,CA_1,2,1.854500,1.854500
10,2016-05-17,CA_1,1,1.516590,1.516590
20,2016-05-18,CA_1,2,1.542208,1.542208
30,2016-05-19,CA_1,1,1.656181,1.656181
40,2016-05-20,CA_1,4,1.863926,1.863926
50,2016-05-21,CA_1,2,2.249851,2.249851
60,2016-05-22,CA_1,5,2.219440,2.219440


In [760]:
for k in tags.keys():
    print(tags[k])
    x_rows = xmat[xmat['unique_id'].isin(tags[k])]
    print(" For ",k," level", rmse(x_rows['y'], x_rows['AutoARIMA']), "-->", rmse(x_rows['y'], x_rows['AutoARIMA/BottomUp']))
    print("\n")
    # print(k+" rmse : "+ str(rmse(x_rows['y'],x_rows['pred_orig']))+"-->"+ str(rmse(x_rows['y'],x_rows['pred_reconciled'])))

['Total']
 For  Country  level 4.8227 --> 4.9561


['CA' 'TX' 'WI']
 For  Country/State  level 2.3888 --> 2.4754


['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
 For  Country/State/Store  level 1.5672 --> 1.5672




# TOP DOWN

In [764]:
reconcilers = [TopDown(method='forecast_proportions')]

hrec = HierarchicalReconciliation(reconcilers=reconcilers)

xhat_rec_td = hrec.reconcile(x_hat,Sums,tags)

In [766]:
evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_hat_df = xhat_rec_td,Y_test_df =x_test,tags = tags , benchmark="AutoARIMA" ).sort_values(by='level')

C:\Users\bhara\anaconda3\Lib\site-packages\hierarchicalforecast\evaluation.py:370: FutureWarning: This class (HierarchicalEvaluation) will be deprecated in future releases. Please use the `hierarchicalforecast.evaluate` function instead.
  warnings.warn(


,level,metric,AutoARIMA,AutoARIMA/TopDown_method-forecast_proportions
0,Country,mse-scaled,1.0,1.000000
1,Country/State,mse-scaled,1.0,0.983070
2,Country/State/Store,mse-scaled,1.0,0.985029
3,Overall,mse-scaled,1.0,0.989874


In [768]:
xmat = pd.merge(left=x_test,right=xhat_rec_td,on=['ds', 'unique_id'])
xmat.columns

Index(['ds', 'unique_id', 'y', 'AutoARIMA',
       'AutoARIMA/TopDown_method-forecast_proportions'],
      dtype='object')

In [770]:
mse_1 = rmse(xmat['y'],xmat['AutoARIMA'])
mse_2 = rmse(xmat['y'],xmat['AutoARIMA/TopDown_method-forecast_proportions'])

print(mse_1,"-->",mse_2)

2.1537 --> 2.1428


In [772]:
for k in tags.keys():
    print(tags[k])

['Total']
['CA' 'TX' 'WI']
['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']


In [774]:
for k in tags.keys():
    r = xmat[xmat['unique_id'].isin(tags[k])]
    print(tags[k])
    print(mse(r["y"],r["AutoARIMA"]))
    print(mse(r["y"],r["AutoARIMA/TopDown_method-forecast_proportions"]))
    print("\n")
    

['Total']
23.258043433083134
23.258043433083134


['CA' 'TX' 'WI']
5.70636321342751
5.609752081150377


['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
2.456268468346513
2.4194968154502368




# MiddleOut

In [777]:
reconcilers = [MiddleOut(middle_level = 'Country/State', top_down_method='forecast_proportions')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

xhat_rec_mo = hrec.reconcile(x_hat,Sums,tags).fillna(0)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_hat_df = xhat_rec_mo,Y_test_df =x_test,tags = tags , benchmark="AutoARIMA" ).sort_values(by='level')

C:\Users\bhara\anaconda3\Lib\site-packages\hierarchicalforecast\evaluation.py:370: FutureWarning: This class (HierarchicalEvaluation) will be deprecated in future releases. Please use the `hierarchicalforecast.evaluate` function instead.
  warnings.warn(


,level,metric,AutoARIMA,AutoARIMA/MiddleOut_middle_level-Country/State_top_down_method-forecast_proportions
0,Country,mse-scaled,1.0,1.016273
1,Country/State,mse-scaled,1.0,1.000000
2,Country/State/Store,mse-scaled,1.0,0.986409
3,Overall,mse-scaled,1.0,1.000688


In [781]:
xmat = pd.merge(left=x_test,right=xhat_rec_mo,on=['ds', 'unique_id'])

mse_1 = rmse(xmat['y'],xmat['AutoARIMA'])
mse_2 = rmse(xmat['y'],xmat['AutoARIMA/MiddleOut_middle_level-Country/State_top_down_method-forecast_proportions'])

In [783]:
for k in tags.keys():
    r = xmat[xmat['unique_id'].isin(tags[k])]
    print(tags[k])
    print(mse(r["y"],r["AutoARIMA"]))
    print(mse(r["y"],r['AutoARIMA/MiddleOut_middle_level-Country/State_top_down_method-forecast_proportions']))
    print("\n")

['Total']
23.258043433083134
23.636528967181807


['CA' 'TX' 'WI']
5.70636321342751
5.70636321342751


['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
2.456268468346513
2.422886390584803




# Min Trace

In [786]:
reconcilers = [MinTrace(method='ols')]
hrec = HierarchicalReconciliation(reconcilers=reconcilers)

xhat_rec_mintrace = hrec.reconcile(x_hat,Sums,tags).fillna(0)

evaluator = HierarchicalEvaluation(evaluators=[mse])
evaluator.evaluate(Y_hat_df = xhat_rec_mintrace, Y_test_df = x_test, tags = tags , benchmark="AutoARIMA" ).sort_values(by='level')

C:\Users\bhara\anaconda3\Lib\site-packages\hierarchicalforecast\evaluation.py:370: FutureWarning: This class (HierarchicalEvaluation) will be deprecated in future releases. Please use the `hierarchicalforecast.evaluate` function instead.
  warnings.warn(


,level,metric,AutoARIMA,AutoARIMA/MinTrace_method-ols
0,Country,mse-scaled,1.0,1.006195
1,Country/State,mse-scaled,1.0,1.004964
2,Country/State/Store,mse-scaled,1.0,0.987400
3,Overall,mse-scaled,1.0,0.998761


In [788]:
xmat = pd.merge(left=x_test,right=xhat_rec_mintrace,on=['ds', 'unique_id'])

mse_1 = rmse(xmat['y'],xmat['AutoARIMA'])
mse_2 = rmse(xmat['y'],xmat['AutoARIMA/MinTrace_method-ols'])

In [790]:
for k in tags.keys():
    r = xmat[xmat['unique_id'].isin(tags[k])]
    print(tags[k])
    print(mse(r["y"],r["AutoARIMA"]))
    print(mse(r["y"],r['AutoARIMA/MinTrace_method-ols']))
    print("\n")

['Total']
23.258043433083134
23.40213730485889


['CA' 'TX' 'WI']
5.70636321342751
5.734688342770467


['CA_1' 'CA_2' 'CA_3' 'CA_4' 'TX_1' 'TX_2' 'TX_3' 'WI_1' 'WI_2' 'WI_3']
2.456268468346513
2.425318353959028


